<a href="https://colab.research.google.com/github/rahiakela/machine-learning-research-and-practice/blob/main/grokking-machine-learning/08-naive-bayes-model/01_spam_detection_with_naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Spam-detection with naive Bayes

Now that we have developed the algorithm, let’s roll up our sleeves and code the naive Bayes
algorithm.

##Setup

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

random.seed(0)

In [ ]:
!wget https://github.com/luisguiserrano/manning/raw/master/Chapter_8_Naive_Bayes/emails.csv

##Dataset preprocessing

First, let's load the dataset.

In [3]:
emails = pd.read_csv("emails.csv")
emails.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [4]:
def process_email(text):
  text = text.lower()
  return list(set(text.split()))

In [5]:
emails["words"] = emails["text"].apply(process_email)
emails.head()

,text,spam,words
0,Subject: naturally irresistible your corporate...,1,"[you, iogo, nowadays, benefits, within, really..."
1,Subject: the stock trading gunslinger fanny i...,1,"[pirogue, earmark, optima, trading, sapling, s..."
2,Subject: unbelievable new homes made easy im ...,1,"[you, show, for, of, minute, unconditionally, ..."
3,Subject: 4 color printing special request add...,1,"[canyon, goldengraphix, for, of, format, ramse..."
4,"Subject: do not have money , get software cds ...",1,"[great, be, money, tradgedies, ain, ?, are, he..."


##Finding the priors

Let’s first find the probability that an email is spam (the prior).

In [6]:
num_emails  = len(emails)
num_spam = sum(emails["spam"])
print(f"Number of emails: {num_emails}")
print(f"Number of spam emails: {num_spam}")
print()

# Calculating the prior probability that an email is spam
print(f"Probability of spam: {num_spam / num_emails}")

Number of emails: 5728
Number of spam emails: 1368

Probability of spam: 0.2388268156424581


We deduce that the prior probability that the email is spam is around 0.24. This is the probability
that an email is spam if we don’t know anything about the email. 

Likewise, the prior probability
that an email is ham is around 0.76.

##Finding the posteriors

We need to find the probabilities that spam (and ham) emails contain a certain word.

In [7]:
#  write a dictionary, and in this dictionary record every word, and its pair of occurrences in spam and ham
model = {}
for index, email in emails.iterrows():
  for word in email["words"]:
    if word not in model:
      # Note that the counts are initialized at 1 to avoid having zero counts
      model[word] = {"spam": 1, "ham": 1}
    if word in model:
      if email["spam"]:
        model[word]["spam"] += 1
      else:
        model[word]["ham"] += 1

Now let’s examine some rows of the dictionary.

In [8]:
model["lottery"]

{'ham': 1, 'spam': 9}

In [9]:
model["sale"]

{'ham': 42, 'spam': 39}

Although this dictionary doesn’t contain any
probabilities, these can be deduced by dividing the first entry by the sum of both entries.


In [13]:
# the probability of lottery being spam
model["lottery"]["spam"] / (model["lottery"]["ham"] + model["lottery"]["spam"])

0.9

In [14]:
# the probability of sale being spam
model["sale"]["spam"] / (model["sale"]["ham"] + model["sale"]["spam"])

0.48148148148148145

Let's generalize it.

In [15]:
def predict_bayes(word):
  word = word.lower()
  num_spam = model[word]["spam"]
  num_ham = model[word]["ham"]
  return 1.0 * num_spam / (num_spam + num_ham)

In [16]:
predict_bayes("lottery")

0.9

In [17]:
predict_bayes("sale")

0.48148148148148145

In [18]:
predict_bayes("won")

0.3595505617977528

##The naive Bayes algorithm